In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy #unsloth_zoo
#!pip install "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"
!pip install sentencepiece protobuf "datasets>=3.4.1"  hf_transfer
!pip install --no-deps unsloth
!pip install --no-deps git+https://github.com/huggingface/transformers.git # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N
!pip install --upgrade unsloth unsloth_zoo huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 24.3 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-mq4_qdto
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-mq4_qdto
  Resolved https://github.com/huggingface/transformers.git to commit 2c0af41ce5c448f872f3222a75f56030fb2e5a88
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.55.0.dev0-py3-none-any.whl size=12136384 sha256=05ad69d39727786fab9a8430ef24c

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_data_formats

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
#!huggingface-cli login

In [ ]:
from PIL import Image
import numpy as np
import random

In [ ]:
def create_dummy_image():
    dummy_array = np.zeros((224, 224, 3), dtype=np.uint8)
    return Image.fromarray(dummy_array)

def create_text_placeholder_image():
    img = Image.new('RGB', (224, 224), color='white')
    from PIL import ImageDraw, ImageFont
    draw = ImageDraw.Draw(img)
    draw.text((50, 100), "PREGUNTA DE TEXTO", fill='black')
    return img

DUMMY_IMAGE = create_dummy_image()

In [ ]:
dataset_name = "sergioq2/coffe"

In [ ]:
dataset_text = load_dataset(dataset_name, split = "train[:5100]")

dataset_coffe.csv:   0%|          | 0.00/638M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5091 [00:00<?, ? examples/s]

In [ ]:
def transform_to_conversations_format(example):
    conversations = [
        {
            "from": "human",
            "value": example["preguntas"]
        },
        {
            "from": "gpt",
            "value": example["respuestas"]
        }
    ]

    return {
        "conversations": conversations,
        "source": "coffe_dataset"
    }

In [ ]:
transformed_dataset_text = dataset_text.map(transform_to_conversations_format)


Map:   0%|          | 0/5091 [00:00<?, ? examples/s]

In [ ]:
columns_to_remove = ["Unnamed: 0", "texto", "preguntas", "respuestas"]
transformed_dataset_text = transformed_dataset_text.remove_columns(columns_to_remove)

In [ ]:
formatted_dataset_text = standardize_data_formats(transformed_dataset_text)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/5091 [00:00<?, ? examples/s]

In [ ]:
def convert_text_to_messages(sample):
    assistant_msg = None

    for conv in sample['conversations']:
        if conv['role'] == 'user':
            user_msg = conv['content']
        elif conv['role'] == 'assistant':
            assistant_msg = conv['content']

    return {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_msg}
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": assistant_msg}
                ]
            },
        ],
        "source": sample.get('source', 'coffe_dataset'),
        "original_text": sample.get('text', '')
    }


In [ ]:
text_dataset_converted = [convert_text_to_messages(s) for s in formatted_dataset_text]
print(f"Text converted: {len(text_dataset_converted)}")

Textos convertidos: 5091


In [ ]:
def add_dummy_image_to_text_dataset(text_dataset):
    modified_dataset = []

    for sample in text_dataset:
        new_sample = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": sample["messages"][0]["content"][0]["text"]},
                        {"type": "image", "image": DUMMY_IMAGE}
                    ]
                },
                {
                    "role": "assistant",
                    "content": sample["messages"][1]["content"]
                }
            ]
        }
        modified_dataset.append(new_sample)

    return modified_dataset


In [ ]:
text_dataset_images = add_dummy_image_to_text_dataset(text_dataset_converted)

In [ ]:
text_dataset_images[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': '¿Qué objetivo tienen los sistemas agroforestales cafeteros según el texto?'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=224x224>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Los sistemas agroforestales cafeteros tienen como objetivo el manejo y la conservación del suelo y el agua, así como el aumento y mantenimiento de la producción, para garantizar la sostenibilidad y el fortalecimiento del desarrollo social y económico de las familias cafeteras.'}]}]}

Function calling

In [ ]:
fc_dataset = "sergioq2/functioncalling_coffedata"

In [ ]:
dataset_fc = load_dataset(fc_dataset, split = "train[:2700]")

functioncalling_dt_coffe.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2702 [00:00<?, ? examples/s]

In [ ]:
dataset_fc

Dataset({
    features: ['query', 'function'],
    num_rows: 2700
})

In [ ]:
def transform_fc_to_conversations_format(example):
    conversations = [
        {
            "from": "human",
            "value": example["query"]
        },
        {
            "from": "gpt",
            "value": example["function"]
        }
    ]
    return {
        "conversations": conversations,
        "source": "function_calling_dataset"
    }

In [ ]:
transformed_dataset_fc = dataset_fc.map(transform_fc_to_conversations_format)

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [ ]:
columns_to_remove_fc = ["query", "function"]
transformed_dataset_fc = transformed_dataset_fc.remove_columns(columns_to_remove_fc)

In [ ]:
formatted_dataset_fc = standardize_data_formats(transformed_dataset_fc)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/2700 [00:00<?, ? examples/s]

In [ ]:
fc_dataset_converted = [convert_text_to_messages(s) for s in formatted_dataset_fc]

In [ ]:
fc_dataset_images = add_dummy_image_to_text_dataset(fc_dataset_converted)

In [ ]:
fc_dataset_images[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Consulta de ingresos: septiembre 2025'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=224x224>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '{"name": "ingresos_consulta", "arguments": "{\\"mes\\": \\"septiembre\\", \\"año\\": \\"2025\\"}"}'}]}]}

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


Image

In [ ]:
import os
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY", "your_roboflow_api_key_here")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)  # Replace with your actual API key
project = rf.workspace("detection-3nbwx").project("coffe-mw9n0")
version = project.version(2)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to coffe-2 in multiclass:: 100%|██████████| 928/928 [00:00<00:00, 8525.14it/s]


In [ ]:
import pandas as pd
from datasets import Dataset, Features, Image, Value
from PIL import Image as PILImage
import os

base_path = '/content/coffe-2/train'
csv_path = os.path.join(base_path, '_classes.csv')

df = pd.read_csv(csv_path)
image_paths = []
texts = []

class_columns = [col.strip() for col in df.columns if col != 'filename']
print(f"Cleaned class names detected: {class_columns}")

class_name_mapping = {
    'Broca': 'Broca',
    'Mancha': 'Mancha de hierro',
    'Ojo': 'Ojo de gallo',
    'Rosado': 'Mal Rosado',
    'Roya': 'Roya'
}

for index, row in df.iterrows():
    filename = row['filename']
    image_full_path = os.path.join(base_path, filename)

    if os.path.exists(image_full_path):
        image_paths.append(image_full_path)

        active_labels = [col.strip() for col, value in row.drop('filename').items() if value == 1]

        mapped_labels = [class_name_mapping.get(label, label) for label in active_labels]

        if mapped_labels:
            if len(mapped_labels) == 1:
                texts.append(f"La enfermedad que tiene la planta es {mapped_labels[0]}")
            else:
                texts.append(f"Las enfermedades que tiene la planta son {', '.join(mapped_labels)}")
        else:
            texts.append("La planta está sana")

    else:
        print(f"Warning: Image not found at path: {image_full_path}. Skipping.")

data = {
    'image': image_paths,
    'text': texts
}

features = Features({
    'image': Image(),
    'text': Value(dtype='string')
})

dataset_images = Dataset.from_dict(data, features=features)

Cleaned class names detected: ['Broca', 'Mancha', 'Ojo', 'Rosado', 'Roya']

Ejemplos de textos generados:
1: La enfermedad que tiene la planta es Ojo de gallo
2: La enfermedad que tiene la planta es Ojo de gallo
3: La enfermedad que tiene la planta es Mancha de hierro
4: La enfermedad que tiene la planta es Roya
5: La enfermedad que tiene la planta es Mal Rosado

Total de imágenes procesadas: 924
Nombres de clases mapeados: ['Broca', 'Mancha de hierro', 'Ojo de gallo', 'Mal Rosado', 'Roya']


In [ ]:
import pandas as pd
from datasets import Dataset, Features, Image, Value
from PIL import Image as PILImage, ImageEnhance, ImageFilter
import random
from collections import Counter

def apply_random_augmentation(image):
    augmentations = [
        lambda img: ImageEnhance.Brightness(img).enhance(random.uniform(0.95, 1.05)),  # ±5% brillo
        lambda img: ImageEnhance.Contrast(img).enhance(random.uniform(0.95, 1.05)),    # ±5% contraste
        lambda img: img.rotate(random.randint(-3, 3), expand=True, fillcolor=(255, 255, 255)),  # ±3 grados
        lambda img: img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 0.3))),  # Blur muy ligero
        lambda img: ImageEnhance.Color(img).enhance(random.uniform(0.98, 1.02)),  # ±2% saturación
        lambda img: ImageEnhance.Sharpness(img).enhance(random.uniform(0.98, 1.02)),   # ±2% nitidez
    ]

    augmentation = random.choice(augmentations)
    return augmentation(image)

def augment_dataset(dataset_images, multiplier_roya_broca=7, multiplier_others=4):
    all_images = []
    all_texts = []

    for i in range(len(dataset_images)):
        original_image = dataset_images[i]['image']
        original_text = dataset_images[i]['text']

        all_images.append(original_image)
        all_texts.append(original_text)

        if "Roya" in original_text or "Broca" in original_text:
            num_augmentations = multiplier_roya_broca - 1
        else:
            num_augmentations = multiplier_others - 1

        if i % 100 == 0:
            print(f"Processing image {i+1}/{len(dataset_images)}")

        for aug_idx in range(num_augmentations):
            try:
                img_copy = original_image.copy()
                augmented_img = apply_random_augmentation(img_copy)
                all_images.append(augmented_img)
                all_texts.append(original_text)
            except Exception as e:
                print(f"Error in image {i}: {e}")
                continue

    new_data = {
        'image': all_images,
        'text': all_texts
    }

    features = Features({
        'image': Image(),
        'text': Value(dtype='string')
    })

    augmented_dataset = Dataset.from_dict(new_data, features=features)

    class_counts = Counter()
    for text in augmented_dataset['text']:
        if "enfermedad que tiene" in text:
            disease = text.split("es ")[-1]
            class_counts[disease] += 1
        elif "enfermedades que tiene" in text:
            diseases = text.split("son ")[-1].split(", ")
            for disease in diseases:
                class_counts[disease.strip()] += 1

    return augmented_dataset

augmented_dataset = augment_dataset(
    dataset_images,
    multiplier_roya_broca=4,
    multiplier_others=2
)

print(f"\nDataset ready: {len(augmented_dataset)} images")

Iniciando augmentación...
Procesando imagen 1/924
Procesando imagen 101/924
Procesando imagen 201/924
Procesando imagen 301/924
Procesando imagen 401/924
Procesando imagen 501/924
Procesando imagen 601/924
Procesando imagen 701/924
Procesando imagen 801/924
Procesando imagen 901/924

Dataset original: 924 imágenes
Dataset augmentado: 2616 imágenes
Factor de aumento: 2.83x

Distribución de clases:
  Ojo de gallo: 360
  Mancha de hierro: 360
  Roya: 816
  Mal Rosado: 360
  Broca: 720

Dataset listo: 2616 imágenes


In [ ]:
def convert_image_to_messages(sample):
    """Convert simple format into messages format"""
    return {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "¿Qué enfermedad tiene esta planta de café?"},
                    {"type": "image", "image": sample["image"]},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": sample["text"]}
                ]
            },
        ],
    }

In [ ]:
image_dataset_converted = [convert_image_to_messages(s) for s in augmented_dataset]
print(f"Images converted: {len(image_dataset_converted)}")

Imágenes convertidas: 2616


In [ ]:
def fix_assistant_content_format(dataset):
    fixed_dataset = []

    for sample in dataset:
        new_sample = sample.copy()

        if isinstance(sample['messages'][1]['content'], list):
            text_content = sample['messages'][1]['content'][0]['text']
            new_sample['messages'][1]['content'] = text_content

        fixed_dataset.append(new_sample)

    return fixed_dataset

In [ ]:
image_dataset_converted = fix_assistant_content_format(image_dataset_converted)
text_dataset_images = fix_assistant_content_format(text_dataset_images)
fc_dataset_images = fix_assistant_content_format(fc_dataset_images)

In [ ]:
image_dataset_converted[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': '¿Qué enfermedad tiene esta planta de café?'},
    {'type': 'image',
     'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=251x185>}]},
  {'role': 'assistant',
   'content': 'La enfermedad que tiene la planta es Ojo de gallo'}]}

In [ ]:
text_dataset_images[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': '¿Qué objetivo tienen los sistemas agroforestales cafeteros según el texto?'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=224x224>}]},
  {'role': 'assistant',
   'content': 'Los sistemas agroforestales cafeteros tienen como objetivo el manejo y la conservación del suelo y el agua, así como el aumento y mantenimiento de la producción, para garantizar la sostenibilidad y el fortalecimiento del desarrollo social y económico de las familias cafeteras.'}]}

In [ ]:
fc_dataset_images[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Consulta de ingresos: septiembre 2025'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=224x224>}]},
  {'role': 'assistant',
   'content': '{"name": "ingresos_consulta", "arguments": "{\\"mes\\": \\"septiembre\\", \\"año\\": \\"2025\\"}"}'}]}

In [ ]:
combined_dataset_full = image_dataset_converted + text_dataset_images + fc_dataset_images
random.shuffle(combined_dataset_full)

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/gemma-3n-E2B-it",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

==((====))==  Unsloth 2025.7.11: Fast Gemma3N patching. Transformers: 4.55.0.dev0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r =32,
    lora_alpha = 64,
    lora_dropout = 0.03,
    bias = "none",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
    target_modules = "all-linear",
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.03.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
import os
os.environ["PYTORCH_CUDA_COMPILE_DISABLE"] = "1"
os.environ["PYTORCH_DISABLE_DYNAMO"] = "1"

In [ ]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    train_dataset=combined_dataset_full,
    processing_class=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        gradient_checkpointing = True,

        gradient_checkpointing_kwargs = {"use_reentrant": False},
        max_grad_norm = 0.3,
        warmup_ratio = 0.05,
        max_steps = 100,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        logging_steps = 1,
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.05,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True}
    )
)

Unsloth: Model does not have a default image size - using 512


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,407 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 42,270,720 of 5,481,708,992 (0.77% trained)


Step,Training Loss
1,8.936200
2,8.577700
3,4.549000
4,3.616900
5,2.889900
6,2.603400
7,2.098400
8,1.870600
9,2.142200
10,1.755800


In [ ]:
import os
HUGGINGFACE_HUB_TOKEN = os.getenv("HUGGINGFACE_HUB_TOKEN", "your_huggingface_token_here")

In [ ]:
if True:
    model.push_to_hub_merged(
        "sergioq2/gemma-3N-finetune-coffe_q4_off", tokenizer,
        token = HUGGINGFACE_HUB_TOKEN
    )

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...netune-coffe_q4_off/tokenizer.model:  96%|#########6| 4.53MB / 4.70MB            

  ...inetune-coffe_q4_off/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00003.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ff/model-00003-of-00003.safetensors:   0%|          | 1.99MB / 2.82GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [03:30<00:00, 70.04s/it]
